In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
spark = SparkSession \
        .builder \
        .master("local[1]") \
        .appName("elt_jobs") \
        .config("spark.sql.files.maxPartitionBytes", "10kb") \
        .getOrCreate()

24/11/27 11:35:40 WARN Utils: Your hostname, MacBook-Pro-de-igor.local resolves to a loopback address: 127.0.0.1; using 172.17.1.206 instead (on interface en0)
24/11/27 11:35:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/27 11:35:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Camada Bronze

In [18]:

df_bronze = spark.read.parquet('DataLake/landing-zone/Bronze/erp/erp_20241127')
df_bronze.show()


+-------------------+--------------------+--------+--------------+
|             curUTC|         guestChecks|  locRef|ingestion_date|
+-------------------+--------------------+--------+--------------+
|2024-05-05T06:06:06|[{NULL, 1234, 109...|99 CB CB|    2024-11-27|
+-------------------+--------------------+--------+--------------+



# Camada Prata

In [21]:
from pyspark.sql.functions import explode, col
df_menu_items = spark.read.parquet('DataLake/landing-zone/Prata/erp/menu_items').show(truncate=False)
df_guest_checks = spark.read.parquet('DataLake/landing-zone/Prata/erp/guest_checks').show(truncate=False)
df_taxes = spark.read.parquet('DataLake/landing-zone/Prata/erp/taxes').show(truncate=False)
df_detail_lines = spark.read.parquet('DataLake/landing-zone/Prata/erp/detail_lines').show(truncate=False)



+-----+-------+---------+-----------+------+--------+-------------+----------+---------+
|miNum|modFlag|inclTax  |activeTaxes|prcLvl|discount|serviceCharge|enderMedia|errorCode|
+-----+-------+---------+-----------+------+--------+-------------+----------+---------+
|6042 |false  |20.809091|28         |3     |NULL    |NULL         |NULL      |NULL     |
+-----+-------+---------+-----------+------+--------+-------------+----------+---------+

+------------+------+----------+-------------------+-------------------+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+------+------+-------------+------+------+------+---------+------+-----+-----+------+-------+------+---------+-----------+
|guestCheckId|chkNum|opnBusDt  |opnUTC             |opnLcl             |clsdBusDt |clsdUTC            |clsdLcl            |lastTransUTC       |lastTransLcl       |lastUpdatedUTC     |lastUpdatedLcl     |clsdFlag|gstCnt|

In [22]:
import sqlite3

def executar_select(tabela: str):
    try:

        conn = sqlite3.connect('erp_database.db')
        cursor = conn.cursor()
        

        select_query = f"SELECT * FROM {tabela} LIMIT 10"  
        cursor.execute(select_query)
        
        resultados = cursor.fetchall()
        

        for linha in resultados:
            print(linha)
        

        conn.close()
        
    except Exception as e:
        print(f"Erro ao executar SELECT: {str(e)}")

# Exemplo de uso
executar_select('menu_items')

('6042', 'False', '20.809091', '28', '3', None, None, None, None)
